## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

### 參考
https://www.kaggle.com/kgkevinwg/monkey-classifier-97-accuracy-using-resnet-50/notebook

https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/

https://www.kaggle.com/kgkevinwg/monkey-classifier-97-accuracy-using-resnet-50/notebook

In [1]:
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
%matplotlib inline
import keras
from keras.applications import VGG16 
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import Adam
from keras.applications.resnet50 import ResNet50
# from keras.models import Model


Using TensorFlow backend.


In [0]:
batch_size = 64 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 30 # 訓練的 epochs 數量

In [3]:
# 讀取 Cifar-10 資料集
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
input_shape = x_train.shape[1:]
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

170500096/170498071 [==============================] - 6s 0us/step
x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [0]:
# 建立 ImageDataGenerator，並指定我們要做資料增強的數值範圍
# rescale 將 image 做標準化
data_generator = ImageDataGenerator(
    rotation_range=20, # 旋轉
    width_shift_range=0.2, # 平移
    height_shift_range=0.2,
    horizontal_flip=True,
    rescale= 1./255)

In [0]:
test_generator = ImageDataGenerator(rescale = 1./255)

In [0]:
data_generator.fit(x_train)

In [8]:
'''
ResNet50 代表我們使用從 imagenet 訓練好的參數來初始
pooling avg 把 feature maps 變成⼀維的向量
include_top 將原本的 Dense layer 拔掉，因為原本這個網路是用來做 1000 個分類的模型，我們必須替換成⾃己的 Dense layer 來符合我們⾃己資料集的類別數量
'''
# 建立 ResNet50 模型
resnet_model = ResNet50(input_shape=input_shape,
                        weights='imagenet',
                        include_top=False)

W0806 13:47:20.124352 140391513520000 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0806 13:47:20.163645 140391513520000 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0806 13:47:20.174381 140391513520000 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0806 13:47:20.218638 140391513520000 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0806 13:47:20.219787

94658560/94653016 [==============================] - 4s 0us/step


In [20]:
# # 建立VGG16模型
# conv_vgg = VGG16(weights = 'imagenet',
#                  include_top = False,
#                  input_shape = input_shape)

58892288/58889256 [==============================] - 2s 0us/step


In [9]:
model = Sequential()
model.add(resnet_model)
model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 1, 1, 2048)        23587712  
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               524544    
_________________________________________________________________
activation_50 (Activation)   (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                2570      
_________________________________________________________________
activation_51 (Activation)   (None, 10)                0         
Total params: 24,114,826
Trainable params: 24,061,706
Non-trainable params: 53,120
___________________________________________________________

In [13]:

'''
已經設定成沒有 Dense layers，且最後⼀層做 GAP
使⽤resnet_model.output 可以取出最後一層的 featuremaps
使⽤ Flatten 攤平後，再接上 Dense layer，神經元數量與資料集的類別數量⼀致
建立模型可以得到一個新的 ResNet-50 模型，且參數是根據 ImageNet 資料集預訓練好的
'''
# last_map = resnet_model.output
# # flatten_map = Flatten()(last_map)
# output = Dense(num_classes)(last_map)

# model = Model(inputs=resnet_model.input, outputs=output)

model.compile(optimizer=Adam(),
                  loss='categorical_crossentropy', metrics=['accuracy'])

model.fit_generator(data_generator.flow(x_train, y_train, batch_size=batch_size),
                        epochs=epochs,
                        validation_data=test_generator.flow(x_test, y_test, batch_size=batch_size),
                        steps_per_epoch=int(len(x_train)//batch_size),
                        validation_steps = 50,
                        workers=4)

# score = model.evaluate(x_test, y_test, verbose=0)
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])

Epoch 1/30
781/781 [==============================] - 219s 281ms/step - loss: 1.4943 - acc: 0.4856 - val_loss: 1.7344 - val_acc: 0.5100
Epoch 2/30
781/781 [==============================] - 199s 254ms/step - loss: 1.1183 - acc: 0.6177 - val_loss: 1.5452 - val_acc: 0.5819
Epoch 3/30
781/781 [==============================] - 197s 253ms/step - loss: 1.1441 - acc: 0.6076 - val_loss: 3.5721 - val_acc: 0.4191
Epoch 4/30
781/781 [==============================] - 198s 253ms/step - loss: 0.9942 - acc: 0.6608 - val_loss: 1.1104 - val_acc: 0.6412
Epoch 5/30
781/781 [==============================] - 197s 252ms/step - loss: 0.9427 - acc: 0.6796 - val_loss: 0.9259 - val_acc: 0.6859
Epoch 6/30
781/781 [==============================] - 200s 255ms/step - loss: 0.8598 - acc: 0.7069 - val_loss: 1.3389 - val_acc: 0.6031
Epoch 7/30
781/781 [==============================] - 195s 249ms/step - loss: 0.8431 - acc: 0.7158 - val_loss: 0.9568 - val_acc: 0.6859
Epoch 8/30
781/781 [============================

In [15]:
score = model.evaluate(x_test/255, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.898408769416809
Test accuracy: 0.7158
